In [2]:
import sys
sys.path.insert(0, '../scripts')
import pipeline
import pandas as pd

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# ML Analysis

We start by exporting the data:

In [9]:
file = '../data/work/block-groups_2012-2016_with-acs_with-q-features.csv'
df = pipeline.read(file)

Defining the train/test sets:

In [10]:
years = [2013, 2014, 2015]
dic_sets = {}
for year in years:
    next_year = year + 1
    dic_sets['train_in_' + str(year)] = (df[df['year']==year], df[df['year']==next_year])

In [11]:
pipeline.columns_list(df)

Index(['GEOID', 'year', 'name', 'parent-location', 'population',
       'poverty-rate', 'renter-occupied-households', 'pct-renter-occupied',
       'median-gross-rent', 'median-household-income',
       ...
       'renter_moved_1980-1989/1970-1979_4quantiles_1.0',
       'renter_moved_1980-1989/1970-1979_4quantiles_2.0',
       'renter_moved_1980-1989/1970-1979_4quantiles_3.0',
       'renter_moved_1980-1989/1970-1979_4quantiles_4.0',
       'renter_moved_1980-1989/1970-1979_4quantiles_nan',
       'renter_moved_1979/1969_earlier_4quantiles_1.0',
       'renter_moved_1979/1969_earlier_4quantiles_2.0',
       'renter_moved_1979/1969_earlier_4quantiles_3.0',
       'renter_moved_1979/1969_earlier_4quantiles_4.0',
       'renter_moved_1979/1969_earlier_4quantiles_nan'],
      dtype='object', length=1003)


Defining the feature columns:

In [ ]:
features = ['population_4quantiles_1.0', 'population_4quantiles_2.0', 'population_4quantiles_3.0', \
            'population_4quantiles_4.0', 'poverty-rate_4quantiles_1.0', 'poverty-rate_4quantiles_2.0', \
            'poverty-rate_4quantiles_3.0', 'poverty-rate_4quantiles_4.0', \
            'renter-occupied-households_4quantiles_1.0', 'renter-occupied-households_4quantiles_2.0', \
            'renter-occupied-households_4quantiles_3.0', 'renter-occupied-households_4quantiles_4.0', \
            'pct-renter-occupied_4quantiles_1.0', 'pct-renter-occupied_4quantiles_2.0', \
            'pct-renter-occupied_4quantiles_3.0', 'pct-renter-occupied_4quantiles_4.0', \
            'median-gross-rent_4quantiles_1.0', 'median-gross-rent_4quantiles_2.0', \
            'median-gross-rent_4quantiles_3.0', 'median-gross-rent_4quantiles_4.0', \
            'median-gross-rent_4quantiles_1.0', 'median-gross-rent_4quantiles_2.0', \
            'median-gross-rent_4quantiles_3.0', 'median-gross-rent_4quantiles_4.0', \
            'median-gross-rent_4quantiles_nan', 'median-household-income_4quantiles_1.0', \
            'median-household-income_4quantiles_2.0', 'median-household-income_4quantiles_3.0', \
            'median-household-income_4quantiles_4.0', 'median-household-income_4quantiles_nan', \
            'median-property-value_4quantiles_1.0', 'median-property-value_4quantiles_2.0', \
            'median-property-value_4quantiles_3.0', 'median-property-value_4quantiles_4.0', \
            'median-property-value_4quantiles_nan', 'rent-burden_4quantiles_1.0', \
            'rent-burden_4quantiles_2.0', 'rent-burden_4quantiles_3.0', 'rent-burden_4quantiles_4.0', \
            'rent-burden_4quantiles_nan', '
            pct-white_4quantiles_1.0	pct-white_4quantiles_2.0	pct-white_4quantiles_3.0	pct-white_4quantiles_4.0
            pct-af-am_4quantiles_1.0	pct-af-am_4quantiles_2.0	pct-af-am_4quantiles_3.0	pct-af-am_4quantiles_4.0
            pct-hispanic_4quantiles_1.0	pct-hispanic_4quantiles_2.0	pct-hispanic_4quantiles_3.0	pct-hispanic_4quantiles_4.0
            pct-am-ind_4quantiles_1.0	pct-am-ind_4quantiles_2.0	pct-am-ind_4quantiles_3.0	pct-am-ind_4quantiles_4.0
            pct-asian_4quantiles_1.0	pct-asian_4quantiles_2.0	pct-asian_4quantiles_3.0	pct-asian_4quantiles_4.0
            pct-nh-pi_4quantiles_1.0	pct-nh-pi_4quantiles_2.0	pct-nh-pi_4quantiles_3.0	pct-nh-pi_4quantiles_4.0
            pct-multiple_4quantiles_1.0	pct-multiple_4quantiles_2.0	pct-multiple_4quantiles_3.0	pct-multiple_4quantiles_4.0
            pct-other_4quantiles_1.0	pct-other_4quantiles_2.0	pct-other_4quantiles_3.0	pct-other_4quantiles_4.0
            
            
            ]

Defining the label: